# Custom Shower Environment
###### Model that gives you a steady, warm shower. How wonderful.

In [64]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [65]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        # Adjusting shower temperature
        self.state += action-1
        
        self.shower_length -= 1
        
        if self.state >= 37 and self.state <=39:
            reward=1
        else:
            reward = -1
        
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [66]:
env = ShowerEnv()

In [67]:
# Trying random actions in the environment

episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("episode and score:", episode, score)
env.close()

episode and score: 1 -60
episode and score: 2 -8
episode and score: 3 -60
episode and score: 4 24
episode and score: 5 -60


In [68]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [69]:
model.learn(total_timesteps=80000)

Logging to Training/Logs/PPO_10
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -27.4    |
| time/              |          |
|    fps             | 1507     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -32.1       |
| time/                   |             |
|    fps                  | 1169        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011126941 |
|    clip_fraction        | 0.0653      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000139   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -17.8       |
| time/                   |             |
|    fps                  | 1054        |
|    iterations           | 11          |
|    time_elapsed         | 21          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.004352579 |
|    clip_fraction        | 0.0521      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.932      |
|    explained_variance   | 0.0036      |
|    learning_rate        | 0.0003      |
|    loss                 | 45.8        |
|    n_updates            | 100         |
|    policy_gradient_loss | 0.000409    |
|    value_loss           | 89.1        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -14.9        |
| time/                   |              |
|    fps                  | 1043         |
|    iterations           | 21           |
|    time_elapsed         | 41           |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0019604587 |
|    clip_fraction        | 0.025        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.909       |
|    explained_variance   | 0.00143      |
|    learning_rate        | 0.0003       |
|    loss                 | 36.4         |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.00172     |
|    value_loss           | 63.2         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 19          |
| time/                   |             |
|    fps                  | 1042        |
|    iterations           | 31          |
|    time_elapsed         | 60          |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.011550881 |
|    clip_fraction        | 0.0943      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.796      |
|    explained_variance   | 0.000599    |
|    learning_rate        | 0.0003      |
|    loss                 | 22.1        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.000552   |
|    value_loss           | 46          |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

In [70]:
shower_path = os.path.join('Training', 'Savel Models', 'Shower_Model_PPO')

In [71]:
model.save(shower_path)

In [72]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = model.predict(state)[0]
        state, reward, done, info = env.step(action)
        score += reward
    print("episode and score:", episode, score)
env.close()

episode and score: 1 50
episode and score: 2 50
episode and score: 3 54
episode and score: 4 52
episode and score: 5 50


As we can see our model gets way better rewards than just trying random actions, like we did in the beginning.

### Enjoy your steady, warm showers.